In [4]:

pip install scikit-learn imbalanced-learn pandas matplotlib plotnine joblib seaborn category-encoders xgboost lightgbm

Note: you may need to restart the kernel to use updated packages.


In [83]:
# Imports
import pandas as pd
import numpy as np
import seaborn as sns
from plotnine import *
import matplotlib.pyplot as plt
import warnings, os, json
from IPython.display import display
from pathlib import Path
from datetime import time, timedelta

In [84]:
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns",None)
pd.set_option("display.width",None)


In [85]:
# PLOT THEME
theme_set(
        theme_minimal()
          + theme(
            figure_size=(18, 8),
            text=element_text(size=11),
            plot_title=element_text(size=16, fontweight="bold"),
            plot_subtitle=element_text(size=12, fontstyle="italic"),
            axis_title= element_text(size=12),
            axis_text=element_text(size=10),
             legend_text=element_text(size=10),
                     legend_position="bottom",
                     )
            )

In [86]:
#Paths
RAW_CSV="streetlightcouncil.csv"
OUT_DIR=Path("processed_data")
OUT_DIR.mkdir(parents=True,exist_ok=True)
ARTIFACT_DIR=Path("artifacts")
ARTIFACT_DIR.mkdir(parents=True, exist_ok=True)

In [87]:
# #DATA LOADING
assaults=pd.read_csv(RAW_CSV)
#load both sheets into a dictionanry of data frames
#sheets =pd.read_excel(assaults, sheet_name=None)

      

In [88]:
assaults


,Reporting_Event_Number,Age,Sex,Race,Ethnicity,Offense_Start_Date,Offense_Code_Name,NIBRS_Code,Case_Status,Division,Offense_Address,Latitude,Longitude,COUNDIST,District_Count
0,8022012875,54.0,Female,Unknown,Unknown,1/1/2018 0:00,RAPE - 1ST DEGREE 510.040 10990 11A,11A,Open/Active (MIG),1,"604 S 3RD ST\r\nLOUISVILLE, KY 40202\r\nUNITED...",38.248468,-85.756551,4.0,307.0
1,8022042334,7.0,Female,White,Unknown,1/1/2018 0:00,RAPE - 1ST DEGREE - VICTIM U/12 YEARS OF AGE 5...,11A,Open/Active (MIG),NaN,"'@CAL\r\nLOUISVILLE, KY 40056\r\nUNITED STATES",NaN,NaN,NaN,NaN
2,8020034673,5.0,Female,Unknown,Unknown,1/1/2018 0:00,SODOMY - 1ST DEGREE - VICTIM U/12 YEARS OF AGE...,11B,Investigation Completed,NaN,"'@CAL\r\nLOUISVILLE, KY 40056\r\nUNITED STATES",NaN,NaN,NaN,NaN
3,8022071938,14.0,Male,Asian,Not Hispanic Or Latino,1/1/2018 0:00,SODOMY - 3RD DEGREE 510.090 11205 11B,11B,Cleared by Arrest,6,"3823 RIMONTE DR\r\nLOUISVILLE, KY 40220\r\nUNI...",38.213538,-85.621250,26.0,52.0
4,8018010966,11.0,Female,White,Unknown,1/1/2018 0:00,SEXUAL ABUSE - 1ST DEGREE- VICTIM U/12 YOA 510...,11D,Closed/Cleared - MIG,3,"7755 BROOKVIEW DR\r\nLOUISVILLE, KY 40214\r\nU...",38.149932,-85.816887,25.0,79.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2265,LMPD23124624,4.0,Female,White,Not Hispanic Or Latino,12/18/2023 8:00,RAPE - 1ST DEGREE - VICTIM U/12 YEARS OF AGE 5...,11A,NaN,7,"7607 ASHTON PARK CIR\r\nLOUISVILLE, KY 40228\r...",38.134511,-85.612740,23.0,41.0
2266,LMPD23125623,4.0,Female,White,Not Hispanic Or Latino,12/19/2023 12:00,SODOMY - 1ST DEGREE - VICTIM U/12 YEARS OF AGE...,11B,NaN,7,"3510 RIPPLE CREEK DR\r\nLOUISVILLE, KY 40229",38.100010,-85.695542,13.0,95.0
2267,LMPD23126445,24.0,Female,Black,Not Hispanic Or Latino,12/22/2023 12:00,RAPE - 1ST DEGREE 510.040 10990 11A,11A,NaN,2,"3911 MIAMI AVE\r\nLOUISVILLE, KY 40212",38.272840,-85.812416,5.0,180.0
2268,LMPD23128103,27.0,Female,White,NaN,12/26/2023 0:00,RAPE - 1ST DEGREE 510.040 10990 11A,11A,NaN,1,"400 S 6TH ST\r\nLOUISVILLE, KY 40202",38.253217,-85.761074,4.0,307.0


In [89]:
# Convert to datetime using the correct column name
assaults['Offense_Start_Date'] = pd.to_datetime(assaults['Offense_Start_Date'], errors='coerce')

# Filter rows where the year is 2023
#assaults = assaults[assaults['Offense Start Date'].dt.year == 2023]


In [90]:
assaults

,Reporting_Event_Number,Age,Sex,Race,Ethnicity,Offense_Start_Date,Offense_Code_Name,NIBRS_Code,Case_Status,Division,Offense_Address,Latitude,Longitude,COUNDIST,District_Count
0,8022012875,54.0,Female,Unknown,Unknown,2018-01-01 00:00:00,RAPE - 1ST DEGREE 510.040 10990 11A,11A,Open/Active (MIG),1,"604 S 3RD ST\r\nLOUISVILLE, KY 40202\r\nUNITED...",38.248468,-85.756551,4.0,307.0
1,8022042334,7.0,Female,White,Unknown,2018-01-01 00:00:00,RAPE - 1ST DEGREE - VICTIM U/12 YEARS OF AGE 5...,11A,Open/Active (MIG),NaN,"'@CAL\r\nLOUISVILLE, KY 40056\r\nUNITED STATES",NaN,NaN,NaN,NaN
2,8020034673,5.0,Female,Unknown,Unknown,2018-01-01 00:00:00,SODOMY - 1ST DEGREE - VICTIM U/12 YEARS OF AGE...,11B,Investigation Completed,NaN,"'@CAL\r\nLOUISVILLE, KY 40056\r\nUNITED STATES",NaN,NaN,NaN,NaN
3,8022071938,14.0,Male,Asian,Not Hispanic Or Latino,2018-01-01 00:00:00,SODOMY - 3RD DEGREE 510.090 11205 11B,11B,Cleared by Arrest,6,"3823 RIMONTE DR\r\nLOUISVILLE, KY 40220\r\nUNI...",38.213538,-85.621250,26.0,52.0
4,8018010966,11.0,Female,White,Unknown,2018-01-01 00:00:00,SEXUAL ABUSE - 1ST DEGREE- VICTIM U/12 YOA 510...,11D,Closed/Cleared - MIG,3,"7755 BROOKVIEW DR\r\nLOUISVILLE, KY 40214\r\nU...",38.149932,-85.816887,25.0,79.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2265,LMPD23124624,4.0,Female,White,Not Hispanic Or Latino,2023-12-18 08:00:00,RAPE - 1ST DEGREE - VICTIM U/12 YEARS OF AGE 5...,11A,NaN,7,"7607 ASHTON PARK CIR\r\nLOUISVILLE, KY 40228\r...",38.134511,-85.612740,23.0,41.0
2266,LMPD23125623,4.0,Female,White,Not Hispanic Or Latino,2023-12-19 12:00:00,SODOMY - 1ST DEGREE - VICTIM U/12 YEARS OF AGE...,11B,NaN,7,"3510 RIPPLE CREEK DR\r\nLOUISVILLE, KY 40229",38.100010,-85.695542,13.0,95.0
2267,LMPD23126445,24.0,Female,Black,Not Hispanic Or Latino,2023-12-22 12:00:00,RAPE - 1ST DEGREE 510.040 10990 11A,11A,NaN,2,"3911 MIAMI AVE\r\nLOUISVILLE, KY 40212",38.272840,-85.812416,5.0,180.0
2268,LMPD23128103,27.0,Female,White,NaN,2023-12-26 00:00:00,RAPE - 1ST DEGREE 510.040 10990 11A,11A,NaN,1,"400 S 6TH ST\r\nLOUISVILLE, KY 40202",38.253217,-85.761074,4.0,307.0


In [91]:
#Calculate basic dataset dimensions
n_rows, n_cols= assaults.shape
# Displays the data set size and information
print("Dataset Dimensions:")
print(f"-Total observations(rows): {n_rows:,}")
print(f"-Total observations(rows): {n_cols:,}")

Dataset Dimensions:
-Total observations(rows): 2,270
-Total observations(rows): 15


In [92]:
#Classify variables by type
all_cols=assaults.columns.tolist()

# Get numeric and categorical variable
all_cols

['Reporting_Event_Number',
 'Age',
 'Sex',
 'Race',
 'Ethnicity',
 'Offense_Start_Date',
 'Offense_Code_Name',
 'NIBRS_Code',
 'Case_Status',
 'Division',
 'Offense_Address',
 'Latitude',
 'Longitude',
 'COUNDIST',
 'District_Count']

In [93]:
assaults.shape

(2270, 15)

In [94]:
#Duplicates
duplicate_count= assaults.duplicated().sum()
display(pd.DataFrame({'Duplicate_Rows':[duplicate_count]}))

,Duplicate_Rows
0,0


In [95]:
assaults.shape

(2270, 15)

In [96]:
assaults = assaults.dropna(subset=['Age', 'COUNDIST'])


In [97]:
#  Convert both columns to integer
assaults['Age'] = assaults['Age'].round().astype(int)
assaults['COUNDIST'] = assaults['COUNDIST'].round().astype(int)


In [98]:
 #convert date from object to datetime
assaults['Offense_Start_Date'] = pd.to_datetime(assaults['Offense_Start_Date'])

#Create unique variables to break apart date and time. 
assaults['date'] = assaults['Offense_Start_Date'].dt.date
assaults['time'] = assaults['Offense_Start_Date'].dt.time


In [99]:
assaults

,Reporting_Event_Number,Age,Sex,Race,Ethnicity,Offense_Start_Date,Offense_Code_Name,NIBRS_Code,Case_Status,Division,Offense_Address,Latitude,Longitude,COUNDIST,District_Count,date,time
0,8022012875,54,Female,Unknown,Unknown,2018-01-01 00:00:00,RAPE - 1ST DEGREE 510.040 10990 11A,11A,Open/Active (MIG),1,"604 S 3RD ST\r\nLOUISVILLE, KY 40202\r\nUNITED...",38.248468,-85.756551,4,307.0,2018-01-01,00:00:00
3,8022071938,14,Male,Asian,Not Hispanic Or Latino,2018-01-01 00:00:00,SODOMY - 3RD DEGREE 510.090 11205 11B,11B,Cleared by Arrest,6,"3823 RIMONTE DR\r\nLOUISVILLE, KY 40220\r\nUNI...",38.213538,-85.621250,26,52.0,2018-01-01,00:00:00
4,8018010966,11,Female,White,Unknown,2018-01-01 00:00:00,SEXUAL ABUSE - 1ST DEGREE- VICTIM U/12 YOA 510...,11D,Closed/Cleared - MIG,3,"7755 BROOKVIEW DR\r\nLOUISVILLE, KY 40214\r\nU...",38.149932,-85.816887,25,79.0,2018-01-01,00:00:00
5,8018046279,5,Male,White,Unknown,2018-01-01 00:00:00,SEXUAL ABUSE - 1ST DEGREE- VICTIM U/12 YOA 510...,11D,Closed/Cleared - MIG,3,"6808 JOHN ADAMS WAY\r\nLOUISVILLE, KY 40272\r\...",38.061941,-85.892340,14,72.0,2018-01-01,00:00:00
6,8018083160,8,Female,White,Hispanic Or Latino,2018-01-01 00:00:00,SEXUAL ABUSE - 1ST DEGREE- VICTIM U/12 YOA 510...,11D,Closed/Cleared - MIG,7,"5112 MILE OF SUNSHINE DR\r\nLOUISVILLE, KY 402...",38.150337,-85.661834,21,135.0,2018-01-01,00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2265,LMPD23124624,4,Female,White,Not Hispanic Or Latino,2023-12-18 08:00:00,RAPE - 1ST DEGREE - VICTIM U/12 YEARS OF AGE 5...,11A,NaN,7,"7607 ASHTON PARK CIR\r\nLOUISVILLE, KY 40228\r...",38.134511,-85.612740,23,41.0,2023-12-18,08:00:00
2266,LMPD23125623,4,Female,White,Not Hispanic Or Latino,2023-12-19 12:00:00,SODOMY - 1ST DEGREE - VICTIM U/12 YEARS OF AGE...,11B,NaN,7,"3510 RIPPLE CREEK DR\r\nLOUISVILLE, KY 40229",38.100010,-85.695542,13,95.0,2023-12-19,12:00:00
2267,LMPD23126445,24,Female,Black,Not Hispanic Or Latino,2023-12-22 12:00:00,RAPE - 1ST DEGREE 510.040 10990 11A,11A,NaN,2,"3911 MIAMI AVE\r\nLOUISVILLE, KY 40212",38.272840,-85.812416,5,180.0,2023-12-22,12:00:00
2268,LMPD23128103,27,Female,White,NaN,2023-12-26 00:00:00,RAPE - 1ST DEGREE 510.040 10990 11A,11A,NaN,1,"400 S 6TH ST\r\nLOUISVILLE, KY 40202",38.253217,-85.761074,4,307.0,2023-12-26,00:00:00


In [100]:
"""
Parse the sunrise and sunset times into one data frame for all years of data (2018-2023). If future years are added, make sure the file format matches 
things to check: make sure data start on column 4 for the date, and time format is hhmm
This code assumes you've gone to https://aa.usno.navy.mil/data/RS_OneYear, entered Louisville and saved a txt file with each year's worth of data individually.
Apologies to future users of this code, I definitely used Claude.ai to help modify Andrew's code that only handled one year of data to include a loop
that iterates through multiple files, one per year. -Angela. 
"""

# Define years and file pattern
years = range(2018, 2024)  # 2018-2023
sun_records = []

for year in years:
    file_path = f"sunset&sunrise_times{year}.txt"
    
    try:
        lines = open(file_path, "r").readlines()
        print(f"Processing {file_path}: {len(lines)} lines found")
        
        records_this_year = 0
        for line in lines:
            # Check if line starts with 2 digits (day of month)
            if len(line.strip()) > 2 and line.strip()[:2].isdigit():
                day = int(line[0:2])
                
                # Each month's rise/set data is in 11-character blocks
                # Format: "hhmm hhmm " (rise and set with space separator)
                col = 4  # Start after "01  " (day number + 2 spaces)
                
                for month in range(12):
                    # Extract the 11-character block for this month
                    block = line[col:col+11]
                    
                    if len(block) >= 9:
                        # Parse rise time (first 4 chars: "hhmm")
                        rise = block[0:4].strip()
                        # Parse set time (chars 5-9: "hhmm")
                        set_ = block[5:9].strip()
                        
                        if rise and set_ and len(rise) == 4 and len(set_) == 4:
                            try:
                                # Convert to time objects
                                sunrise = time(int(rise[:2]), int(rise[2:]))
                                sunset = time(int(set_[:2]), int(set_[2:]))
                                
                                # Create date with proper year
                                date = pd.to_datetime(
                                    f"{year}-{month+1:02d}-{day:02d}"
                                ).date()
                                
                                sun_records.append({
                                    "Date": date,
                                    "Sunrise": sunrise,
                                    "Sunset": sunset
                                })
                                records_this_year += 1
                            except (ValueError, IndexError):
                                # Skip malformed time entries
                                pass
                    
                    col += 11  # Move to next month's block
        
        print(f"  -> Added {records_this_year} records from {year}")
    
    except FileNotFoundError:
        print(f"Warning: File {file_path} not found. Skipping {year}.")
        continue
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        continue

# Create single DataFrame with all years
if sun_records:
    df_sun = pd.DataFrame(sun_records)
    df_sun = df_sun.sort_values("Date").reset_index(drop=True)
    
    print(f"\n{'='*60}")
    print(f"Total records: {len(df_sun)}")
    print(f"Date range: {df_sun['Date'].min()} to {df_sun['Date'].max()}")
    print(f"\nFirst few records:")
    print(df_sun.head(10))
    print(f"\nLast few records:")
    print(df_sun.tail(10))
else:
    print("\nERROR: No records were parsed from any files.")
    print("Please check file format and naming.")




Processing sunset&sunrise_times2018.txt: 42 lines found
  -> Added 365 records from 2018
Processing sunset&sunrise_times2019.txt: 42 lines found
  -> Added 365 records from 2019
Processing sunset&sunrise_times2020.txt: 42 lines found
  -> Added 366 records from 2020
Processing sunset&sunrise_times2021.txt: 42 lines found
  -> Added 365 records from 2021
Processing sunset&sunrise_times2022.txt: 42 lines found
  -> Added 365 records from 2022
Processing sunset&sunrise_times2023.txt: 42 lines found
  -> Added 365 records from 2023

Total records: 2191
Date range: 2018-01-01 to 2023-12-31

First few records:
         Date   Sunrise    Sunset
0  2018-01-01  08:00:00  17:34:00
1  2018-01-02  08:00:00  17:35:00
2  2018-01-03  08:00:00  17:35:00
3  2018-01-04  08:00:00  17:36:00
4  2018-01-05  08:00:00  17:37:00
5  2018-01-06  08:00:00  17:38:00
6  2018-01-07  08:00:00  17:39:00
7  2018-01-08  08:00:00  17:40:00
8  2018-01-09  08:00:00  17:41:00
9  2018-01-10  08:00:00  17:42:00

Last few reco

In [101]:
#Next we have to adjust for daylight savings because the naval observatory data does not include that in its tables. Again, more vibe coding. -Angela

def adjust_for_dst(df, dst_file_path="DST.txt"):
    """
    Adjust sunrise and sunset times for Daylight Saving Time.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame with columns: Date, Sunrise, Sunset
    dst_file_path : str
        Path to the DST periods file
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame with DST-adjusted times
    """
    # Make a copy to avoid modifying the original
    df_adjusted = df.copy()
    
    # Read DST periods file
    dst_periods = []
    with open(dst_file_path, "r") as f:
        lines = f.readlines()
        
    i = 0
    while i < len(lines):
        line = lines[i].strip()
        if line.startswith("Begin,"):
            begin_date = pd.to_datetime(line.split(", ")[1]).date()
            # Next line should be End
            if i + 1 < len(lines):
                end_line = lines[i + 1].strip()
                if end_line.startswith("End,"):
                    end_date = pd.to_datetime(end_line.split(", ")[1]).date()
                    dst_periods.append((begin_date, end_date))
                    i += 2
                    continue
        i += 1
    
    print(f"Found {len(dst_periods)} DST periods:")
    for begin, end in dst_periods:
        print(f"  {begin} to {end}")
    
    # Function to add one hour to a time object
    def add_hour(t):
        dt = pd.Timestamp.combine(pd.Timestamp.today(), t)
        dt_plus_hour = dt + timedelta(hours=1)
        return dt_plus_hour.time()
    
    # Apply DST adjustment
    adjusted_count = 0
    for begin_date, end_date in dst_periods:
        # Create mask for dates in DST period
        mask = (df_adjusted['Date'] >= begin_date) & (df_adjusted['Date'] <= end_date)
        
        # Add one hour to sunrise and sunset for dates in DST
        df_adjusted.loc[mask, 'Sunrise'] = df_adjusted.loc[mask, 'Sunrise'].apply(add_hour)
        df_adjusted.loc[mask, 'Sunset'] = df_adjusted.loc[mask, 'Sunset'].apply(add_hour)
        
        adjusted_count += mask.sum()


    print(f"\nAdjusted {adjusted_count} records for DST")
    
    return df_adjusted



df_sun_adjusted = adjust_for_dst(df_sun, "DST.txt")

# To see the difference, you can compare before and after:
print("\nBefore DST adjustment:")
print(df_sun[df_sun['Date'] == pd.to_datetime('2018-03-11').date()])
print("\nAfter DST adjustment:")
print(df_sun_adjusted[df_sun_adjusted['Date'] == pd.to_datetime('2018-03-11').date()])


#Now we have two data frames we can work with. 
#df_sun is the original parsed file that has no changes or adjustments for daylight savings.
#df_sun_adjusted has the correction for daylight savings. 
#I'm keeping both in the code in case we find any issue with using DST in the future, for example, if LMPD records error in the future and dont actually adjust.
#Moving forward the analysis will use df_sun_adjusted.



Found 6 DST periods:
  2018-03-11 to 2018-11-04
  2019-03-10 to 2019-11-03
  2020-03-08 to 2020-11-01
  2021-03-14 to 2021-11-07
  2022-03-13 to 2022-11-06
  2023-03-12 to 2023-11-05

Adjusted 1434 records for DST

Before DST adjustment:
          Date   Sunrise    Sunset
69  2018-03-11  07:00:00  18:46:00

After DST adjustment:
          Date   Sunrise    Sunset
69  2018-03-11  08:00:00  19:46:00


In [102]:
df_sun_adjusted

,Date,Sunrise,Sunset
0,2018-01-01,08:00:00,17:34:00
1,2018-01-02,08:00:00,17:35:00
2,2018-01-03,08:00:00,17:35:00
3,2018-01-04,08:00:00,17:36:00
4,2018-01-05,08:00:00,17:37:00
...,...,...,...
2186,2023-12-27,05:57:00,18:05:00
2187,2023-12-28,05:58:00,18:05:00
2188,2023-12-29,05:58:00,18:06:00
2189,2023-12-30,05:59:00,18:06:00


In [103]:
assaults

,Reporting_Event_Number,Age,Sex,Race,Ethnicity,Offense_Start_Date,Offense_Code_Name,NIBRS_Code,Case_Status,Division,Offense_Address,Latitude,Longitude,COUNDIST,District_Count,date,time
0,8022012875,54,Female,Unknown,Unknown,2018-01-01 00:00:00,RAPE - 1ST DEGREE 510.040 10990 11A,11A,Open/Active (MIG),1,"604 S 3RD ST\r\nLOUISVILLE, KY 40202\r\nUNITED...",38.248468,-85.756551,4,307.0,2018-01-01,00:00:00
3,8022071938,14,Male,Asian,Not Hispanic Or Latino,2018-01-01 00:00:00,SODOMY - 3RD DEGREE 510.090 11205 11B,11B,Cleared by Arrest,6,"3823 RIMONTE DR\r\nLOUISVILLE, KY 40220\r\nUNI...",38.213538,-85.621250,26,52.0,2018-01-01,00:00:00
4,8018010966,11,Female,White,Unknown,2018-01-01 00:00:00,SEXUAL ABUSE - 1ST DEGREE- VICTIM U/12 YOA 510...,11D,Closed/Cleared - MIG,3,"7755 BROOKVIEW DR\r\nLOUISVILLE, KY 40214\r\nU...",38.149932,-85.816887,25,79.0,2018-01-01,00:00:00
5,8018046279,5,Male,White,Unknown,2018-01-01 00:00:00,SEXUAL ABUSE - 1ST DEGREE- VICTIM U/12 YOA 510...,11D,Closed/Cleared - MIG,3,"6808 JOHN ADAMS WAY\r\nLOUISVILLE, KY 40272\r\...",38.061941,-85.892340,14,72.0,2018-01-01,00:00:00
6,8018083160,8,Female,White,Hispanic Or Latino,2018-01-01 00:00:00,SEXUAL ABUSE - 1ST DEGREE- VICTIM U/12 YOA 510...,11D,Closed/Cleared - MIG,7,"5112 MILE OF SUNSHINE DR\r\nLOUISVILLE, KY 402...",38.150337,-85.661834,21,135.0,2018-01-01,00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2265,LMPD23124624,4,Female,White,Not Hispanic Or Latino,2023-12-18 08:00:00,RAPE - 1ST DEGREE - VICTIM U/12 YEARS OF AGE 5...,11A,NaN,7,"7607 ASHTON PARK CIR\r\nLOUISVILLE, KY 40228\r...",38.134511,-85.612740,23,41.0,2023-12-18,08:00:00
2266,LMPD23125623,4,Female,White,Not Hispanic Or Latino,2023-12-19 12:00:00,SODOMY - 1ST DEGREE - VICTIM U/12 YEARS OF AGE...,11B,NaN,7,"3510 RIPPLE CREEK DR\r\nLOUISVILLE, KY 40229",38.100010,-85.695542,13,95.0,2023-12-19,12:00:00
2267,LMPD23126445,24,Female,Black,Not Hispanic Or Latino,2023-12-22 12:00:00,RAPE - 1ST DEGREE 510.040 10990 11A,11A,NaN,2,"3911 MIAMI AVE\r\nLOUISVILLE, KY 40212",38.272840,-85.812416,5,180.0,2023-12-22,12:00:00
2268,LMPD23128103,27,Female,White,NaN,2023-12-26 00:00:00,RAPE - 1ST DEGREE 510.040 10990 11A,11A,NaN,1,"400 S 6TH ST\r\nLOUISVILLE, KY 40202",38.253217,-85.761074,4,307.0,2023-12-26,00:00:00


In [104]:
#convert date from object to datetime
df_sun_adjusted['Date'] = pd.to_datetime(df_sun_adjusted['Date'])
assaults['date'] = pd.to_datetime(assaults['date'])
                                  
# Merge sunrise/sunset into assault data
sun_df = pd.DataFrame(df_sun_adjusted)
assaults = pd.merge(
    left=assaults,
    right=sun_df,
    how='left', 
    left_on='date', 
    right_on='Date'    
)  


In [105]:

#Flag nighttime incidents (Sunset → Sunrise)
assaults['Is_Nighttime'] = (
    (assaults['time'] >= assaults['Sunset']) |
    (assaults['time'] < assaults['Sunrise'])
)

In [106]:

# Filter for nighttime incidents

night_df = assaults[
    (assaults['Is_Nighttime'])
]

#drop the extra date column left over from the merge.
night_df.drop(['Date'], axis=1, inplace=True)


In [107]:
night_df.shape


(992, 20)

In [108]:
night_df

,Reporting_Event_Number,Age,Sex,Race,Ethnicity,Offense_Start_Date,Offense_Code_Name,NIBRS_Code,Case_Status,Division,Offense_Address,Latitude,Longitude,COUNDIST,District_Count,date,time,Sunrise,Sunset,Is_Nighttime
0,8022012875,54,Female,Unknown,Unknown,2018-01-01 00:00:00,RAPE - 1ST DEGREE 510.040 10990 11A,11A,Open/Active (MIG),1,"604 S 3RD ST\r\nLOUISVILLE, KY 40202\r\nUNITED...",38.248468,-85.756551,4,307.0,2018-01-01,00:00:00,08:00:00,17:34:00,True
1,8022071938,14,Male,Asian,Not Hispanic Or Latino,2018-01-01 00:00:00,SODOMY - 3RD DEGREE 510.090 11205 11B,11B,Cleared by Arrest,6,"3823 RIMONTE DR\r\nLOUISVILLE, KY 40220\r\nUNI...",38.213538,-85.621250,26,52.0,2018-01-01,00:00:00,08:00:00,17:34:00,True
2,8018010966,11,Female,White,Unknown,2018-01-01 00:00:00,SEXUAL ABUSE - 1ST DEGREE- VICTIM U/12 YOA 510...,11D,Closed/Cleared - MIG,3,"7755 BROOKVIEW DR\r\nLOUISVILLE, KY 40214\r\nU...",38.149932,-85.816887,25,79.0,2018-01-01,00:00:00,08:00:00,17:34:00,True
3,8018046279,5,Male,White,Unknown,2018-01-01 00:00:00,SEXUAL ABUSE - 1ST DEGREE- VICTIM U/12 YOA 510...,11D,Closed/Cleared - MIG,3,"6808 JOHN ADAMS WAY\r\nLOUISVILLE, KY 40272\r\...",38.061941,-85.892340,14,72.0,2018-01-01,00:00:00,08:00:00,17:34:00,True
4,8018083160,8,Female,White,Hispanic Or Latino,2018-01-01 00:00:00,SEXUAL ABUSE - 1ST DEGREE- VICTIM U/12 YOA 510...,11D,Closed/Cleared - MIG,7,"5112 MILE OF SUNSHINE DR\r\nLOUISVILLE, KY 402...",38.150337,-85.661834,21,135.0,2018-01-01,00:00:00,08:00:00,17:34:00,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2002,LMPD23123249,16,Female,White,Hispanic Or Latino,2023-12-14 03:50:00,RAPE - 1ST DEGREE 510.040 10990 11A,11A,Open/Active,6,"1367 GARDINER LN\r\nWATTERSON PARK, KY 40213",38.197717,-85.700115,10,91.0,2023-12-14,03:50:00,05:51:00,17:58:00,True
2003,LMPD23129385,18,Female,White,Hispanic Or Latino,2023-12-15 00:00:00,RAPE - 1ST DEGREE 510.040 10990 11A,11A,NaN,7,"SIXMILE ISLAND\r\nLOUISVILLE, KY 99999\r\nUNIT...",38.170449,-85.603572,26,52.0,2023-12-15,00:00:00,05:51:00,17:59:00,True
2004,LMPD23124681,31,Female,White,Not Hispanic Or Latino,2023-12-15 22:00:00,RAPE - 1ST DEGREE 510.040 10990 11A,11A,SVU Only - Victim Requests Closure,2,"4713 BEECH DR\r\nLOUISVILLE, KY 40216",38.202771,-85.844656,1,100.0,2023-12-15,22:00:00,05:51:00,17:59:00,True
2008,LMPD23128103,27,Female,White,NaN,2023-12-26 00:00:00,RAPE - 1ST DEGREE 510.040 10990 11A,11A,NaN,1,"400 S 6TH ST\r\nLOUISVILLE, KY 40202",38.253217,-85.761074,4,307.0,2023-12-26,00:00:00,05:57:00,18:04:00,True


In [50]:
"""
next working with the open data portal data set to merge in the location variable
download from https://data.louisvilleky.gov/ Louisville Metro Crime Data.  Years 2018-2023. Rename files to crime_yyyy.csv
merge variable is going to be 'reporting_event_number' in our current night_df. In open data it's 'INCIDENT_NUMBER'. We need to bring in the
'PREMISE_TYPE' variable from open data to be able to determine whether the assault happened somewhere outside.  

Some records are 'LMPDyyNNNNNN' and others use the format '80yyNNNNNN' where 'yy' is the last two digits of the year and NNNNNN is the report number. 
"""


In [55]:
#standardizing the Reporting_Event_Number values

#night_df['Reporting_Event_Number'] = night_df['Reporting_Event_Number'].str.replace('^LMPD', '80', regex=True)

In [109]:
night_df

,Reporting_Event_Number,Age,Sex,Race,Ethnicity,Offense_Start_Date,Offense_Code_Name,NIBRS_Code,Case_Status,Division,Offense_Address,Latitude,Longitude,COUNDIST,District_Count,date,time,Sunrise,Sunset,Is_Nighttime
0,8022012875,54,Female,Unknown,Unknown,2018-01-01 00:00:00,RAPE - 1ST DEGREE 510.040 10990 11A,11A,Open/Active (MIG),1,"604 S 3RD ST\r\nLOUISVILLE, KY 40202\r\nUNITED...",38.248468,-85.756551,4,307.0,2018-01-01,00:00:00,08:00:00,17:34:00,True
1,8022071938,14,Male,Asian,Not Hispanic Or Latino,2018-01-01 00:00:00,SODOMY - 3RD DEGREE 510.090 11205 11B,11B,Cleared by Arrest,6,"3823 RIMONTE DR\r\nLOUISVILLE, KY 40220\r\nUNI...",38.213538,-85.621250,26,52.0,2018-01-01,00:00:00,08:00:00,17:34:00,True
2,8018010966,11,Female,White,Unknown,2018-01-01 00:00:00,SEXUAL ABUSE - 1ST DEGREE- VICTIM U/12 YOA 510...,11D,Closed/Cleared - MIG,3,"7755 BROOKVIEW DR\r\nLOUISVILLE, KY 40214\r\nU...",38.149932,-85.816887,25,79.0,2018-01-01,00:00:00,08:00:00,17:34:00,True
3,8018046279,5,Male,White,Unknown,2018-01-01 00:00:00,SEXUAL ABUSE - 1ST DEGREE- VICTIM U/12 YOA 510...,11D,Closed/Cleared - MIG,3,"6808 JOHN ADAMS WAY\r\nLOUISVILLE, KY 40272\r\...",38.061941,-85.892340,14,72.0,2018-01-01,00:00:00,08:00:00,17:34:00,True
4,8018083160,8,Female,White,Hispanic Or Latino,2018-01-01 00:00:00,SEXUAL ABUSE - 1ST DEGREE- VICTIM U/12 YOA 510...,11D,Closed/Cleared - MIG,7,"5112 MILE OF SUNSHINE DR\r\nLOUISVILLE, KY 402...",38.150337,-85.661834,21,135.0,2018-01-01,00:00:00,08:00:00,17:34:00,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2002,LMPD23123249,16,Female,White,Hispanic Or Latino,2023-12-14 03:50:00,RAPE - 1ST DEGREE 510.040 10990 11A,11A,Open/Active,6,"1367 GARDINER LN\r\nWATTERSON PARK, KY 40213",38.197717,-85.700115,10,91.0,2023-12-14,03:50:00,05:51:00,17:58:00,True
2003,LMPD23129385,18,Female,White,Hispanic Or Latino,2023-12-15 00:00:00,RAPE - 1ST DEGREE 510.040 10990 11A,11A,NaN,7,"SIXMILE ISLAND\r\nLOUISVILLE, KY 99999\r\nUNIT...",38.170449,-85.603572,26,52.0,2023-12-15,00:00:00,05:51:00,17:59:00,True
2004,LMPD23124681,31,Female,White,Not Hispanic Or Latino,2023-12-15 22:00:00,RAPE - 1ST DEGREE 510.040 10990 11A,11A,SVU Only - Victim Requests Closure,2,"4713 BEECH DR\r\nLOUISVILLE, KY 40216",38.202771,-85.844656,1,100.0,2023-12-15,22:00:00,05:51:00,17:59:00,True
2008,LMPD23128103,27,Female,White,NaN,2023-12-26 00:00:00,RAPE - 1ST DEGREE 510.040 10990 11A,11A,NaN,1,"400 S 6TH ST\r\nLOUISVILLE, KY 40202",38.253217,-85.761074,4,307.0,2023-12-26,00:00:00,05:57:00,18:04:00,True


In [110]:
import os

# Define path to your Documents folder
documents_path = os.path.expanduser("~/Documents")

# Define filename
filename = "night_df.csv"

# Full path
full_path = os.path.join(documents_path, filename)

# Save DataFrame
night_df.to_csv(full_path, index=False)

print(f"night_df to: {full_path}")


night_df to: C:\Users\ajgraham/Documents\night_df.csv


In [111]:
#buckle up, here's more vibe coding. 
#we're merging the open data portal records into one data frame for all years keeping only the incident number and premise type.
#this also reformats the incident number to remove hyphens.
#make sure all the file column headers are the same before you run this. in 2023 they changed to lowercase and instead of 'PREMISE_TYPE' it's location_category' 
#I changed those columns in the csv file before I ran this code.

# Define years
years = range(2018, 2024)  # 2018-2023

# List to store dataframes
crime_dfs = []

for year in years:
    file_path = f"crime_{year}.csv"
    
    try:
        # Read only the columns we need
        df = pd.read_csv(file_path, usecols=['INCIDENT_NUMBER', 'PREMISE_TYPE'])
        
        print(f"Loaded {file_path}: {len(df)} records")
        crime_dfs.append(df)
        
    except FileNotFoundError:
        print(f"Warning: {file_path} not found. Skipping.")
        continue
    except KeyError as e:
        print(f"Error: Column not found in {file_path}: {e}")
        continue

# Combine all dataframes
if crime_dfs:
    crime_df = pd.concat(crime_dfs, ignore_index=True)
    
    # Remove hyphens from INCIDENT_NUMBER
    crime_df['INCIDENT_NUMBER'] = crime_df['INCIDENT_NUMBER'].str.replace('-', '', regex=False)
    
    #reformat anything that is starting with LMPD to start with 80 instead to standardize things.
    #crime_df['INCIDENT_NUMBER'] = crime_df['INCIDENT_NUMBER'].str.replace('^LMPD', '80', regex=True)

    
    print(f"\n{'='*60}")
    print(f"Total records combined: {len(crime_df)}")
    print(f"\nFirst few records:")
    print(crime_df.head(10))
    print(f"\nDataFrame info:")
    print(crime_df.info())
else:
    print("ERROR: No crime files were successfully loaded.")


Loaded crime_2018.csv: 80000 records
Loaded crime_2019.csv: 73420 records
Loaded crime_2020.csv: 70233 records
Loaded crime_2021.csv: 72127 records
Loaded crime_2022.csv: 71506 records
Loaded crime_2023.csv: 71435 records

Total records combined: 438721

First few records:
  INCIDENT_NUMBER                        PREMISE_TYPE
0      8018023451  NON-ATTACHED RESD GARAGE/SHED/BULD
1      8018026875      SPECIALTY STORE (TV, FUR, ETC)
2      8018084991              HIGHWAY / ROAD / ALLEY
3      8018023451  NON-ATTACHED RESD GARAGE/SHED/BULD
4      8018025979              HIGHWAY / ROAD / ALLEY
5      8018084991              HIGHWAY / ROAD / ALLEY
6      8018026876         CHURCH / SYNAGOGUE / TEMPLE
7      8018023441                     OTHER / UNKNOWN
8      8018023218                    RESIDENCE / HOME
9      8018025376              HIGHWAY / ROAD / ALLEY

DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438721 entries, 0 to 438720
Data columns (total 2 columns):
 #   

In [122]:
crime_df

,INCIDENT_NUMBER,PREMISE_TYPE
0,8018023451,NON-ATTACHED RESD GARAGE/SHED/BULD
1,8018026875,"SPECIALTY STORE (TV, FUR, ETC)"
2,8018084991,HIGHWAY / ROAD / ALLEY
3,8018023451,NON-ATTACHED RESD GARAGE/SHED/BULD
4,8018025979,HIGHWAY / ROAD / ALLEY
...,...,...
438716,8023000024,PARKING/ DROP LOT/ GARAGE
438717,8023500224,RESIDENCE/HOME
438718,8023000086,RESIDENCE/HOME
438719,8023000007,HIGHWAY/ ROAD/ ALLEY/ STREET/ SIDEWALK


In [120]:
#Duplicates
#Filter the original DataFrame to create a new one with only the duplicates.  After merging the open data info, there is quite a lot of duplication.
#This bit of code lets you inspect the duplicates and is optional. 
df_duplicates = crime_df[crime_df.duplicated(subset='INCIDENT_NUMBER', keep=False)]

print("\nDataFrame containing only duplicate rows:")
df_duplicates



DataFrame containing only duplicate rows:


,INCIDENT_NUMBER,PREMISE_TYPE
0,8018023451,NON-ATTACHED RESD GARAGE/SHED/BULD
2,8018084991,HIGHWAY / ROAD / ALLEY
3,8018023451,NON-ATTACHED RESD GARAGE/SHED/BULD
5,8018084991,HIGHWAY / ROAD / ALLEY
7,8018023441,OTHER / UNKNOWN
...,...,...
438710,8023000012,RESIDENCE/HOME
438711,8023000012,RESIDENCE/HOME
438712,8023000012,RESIDENCE/HOME
438715,8023000024,HIGHWAY/ ROAD/ ALLEY/ STREET/ SIDEWALK


In [121]:
import os

# Define path to your Documents folder
documents_path = os.path.expanduser("~/Documents")

# Define filename
filename = "dupes.csv"

# Full path
full_path = os.path.join(documents_path, filename)

# Save DataFrame
df_duplicates.to_csv(full_path, index=False)

print(f"df_duplicates to: {full_path}")

df_duplicates to: C:\Users\ajgraham/Documents\dupes.csv


In [125]:
#Merge the premise type into the original data set using incident number. With python's left join it will duplicate rows if there is a one to many match.
#see notes above about why we have a one to many match. 

crime_df = pd.DataFrame(crime_df)
night_loc_df = pd.merge(
    left=night_df,
    right=crime_df,
    how='left', 
    left_on='Reporting_Event_Number', 
    right_on='INCIDENT_NUMBER'    
)  

#dropped the extra 'incident number' column. 
night_loc_df.drop(['INCIDENT_NUMBER'], axis=1, inplace=True)

In [126]:
night_loc_df

,Reporting_Event_Number,Age,Sex,Race,Ethnicity,Offense_Start_Date,Offense_Code_Name,NIBRS_Code,Case_Status,Division,Offense_Address,Latitude,Longitude,COUNDIST,District_Count,date,time,Sunrise,Sunset,Is_Nighttime,PREMISE_TYPE
0,8022012875,54,Female,Unknown,Unknown,2018-01-01 00:00:00,RAPE - 1ST DEGREE 510.040 10990 11A,11A,Open/Active (MIG),1,"604 S 3RD ST\r\nLOUISVILLE, KY 40202\r\nUNITED...",38.248468,-85.756551,4,307.0,2018-01-01,00:00:00,08:00:00,17:34:00,True,RESIDENCE / HOME
1,8022071938,14,Male,Asian,Not Hispanic Or Latino,2018-01-01 00:00:00,SODOMY - 3RD DEGREE 510.090 11205 11B,11B,Cleared by Arrest,6,"3823 RIMONTE DR\r\nLOUISVILLE, KY 40220\r\nUNI...",38.213538,-85.621250,26,52.0,2018-01-01,00:00:00,08:00:00,17:34:00,True,NaN
2,8018010966,11,Female,White,Unknown,2018-01-01 00:00:00,SEXUAL ABUSE - 1ST DEGREE- VICTIM U/12 YOA 510...,11D,Closed/Cleared - MIG,3,"7755 BROOKVIEW DR\r\nLOUISVILLE, KY 40214\r\nU...",38.149932,-85.816887,25,79.0,2018-01-01,00:00:00,08:00:00,17:34:00,True,RESIDENCE / HOME
3,8018046279,5,Male,White,Unknown,2018-01-01 00:00:00,SEXUAL ABUSE - 1ST DEGREE- VICTIM U/12 YOA 510...,11D,Closed/Cleared - MIG,3,"6808 JOHN ADAMS WAY\r\nLOUISVILLE, KY 40272\r\...",38.061941,-85.892340,14,72.0,2018-01-01,00:00:00,08:00:00,17:34:00,True,RESIDENCE / HOME
4,8018083160,8,Female,White,Hispanic Or Latino,2018-01-01 00:00:00,SEXUAL ABUSE - 1ST DEGREE- VICTIM U/12 YOA 510...,11D,Closed/Cleared - MIG,7,"5112 MILE OF SUNSHINE DR\r\nLOUISVILLE, KY 402...",38.150337,-85.661834,21,135.0,2018-01-01,00:00:00,08:00:00,17:34:00,True,RESIDENCE / HOME
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1339,LMPD23123249,16,Female,White,Hispanic Or Latino,2023-12-14 03:50:00,RAPE - 1ST DEGREE 510.040 10990 11A,11A,Open/Active,6,"1367 GARDINER LN\r\nWATTERSON PARK, KY 40213",38.197717,-85.700115,10,91.0,2023-12-14,03:50:00,05:51:00,17:58:00,True,HOTEL/ MOTEL/ ETC.
1340,LMPD23129385,18,Female,White,Hispanic Or Latino,2023-12-15 00:00:00,RAPE - 1ST DEGREE 510.040 10990 11A,11A,NaN,7,"SIXMILE ISLAND\r\nLOUISVILLE, KY 99999\r\nUNIT...",38.170449,-85.603572,26,52.0,2023-12-15,00:00:00,05:51:00,17:59:00,True,OTHER/UNKNOWN
1341,LMPD23124681,31,Female,White,Not Hispanic Or Latino,2023-12-15 22:00:00,RAPE - 1ST DEGREE 510.040 10990 11A,11A,SVU Only - Victim Requests Closure,2,"4713 BEECH DR\r\nLOUISVILLE, KY 40216",38.202771,-85.844656,1,100.0,2023-12-15,22:00:00,05:51:00,17:59:00,True,RESIDENCE/HOME
1342,LMPD23128103,27,Female,White,NaN,2023-12-26 00:00:00,RAPE - 1ST DEGREE 510.040 10990 11A,11A,NaN,1,"400 S 6TH ST\r\nLOUISVILLE, KY 40202",38.253217,-85.761074,4,307.0,2023-12-26,00:00:00,05:57:00,18:04:00,True,JAIL/PRISON/PENITENTIARY/CORRECTIONS FACILITY


In [ ]:
"""
We have a data set now with:
-all charges associated with the incidents identified by LMPD in the original data request
-events filtered to nighttime hours 
-premise type from the open data records 

I have not created a flag for "outdoor" or incidents that have occured somewhere that may be influenced by the built environment. Andrew did this for 
his analysis originally, but we might want to change our definitions based on literature reviews or discussions with stakeholders.

We also need to determine the best way to handle counts, mapping, and analysis given the fact that we have duplicate LMPD incident numbers, so we dont count 
the same incident multiple times. 

"""


In [128]:
#Exporting our final data set to a CSV.
# Define path to your Documents folder
documents_path = os.path.expanduser("~/Documents")

# Define filename
filename = "sexualassault_2018-2023.csv"

# Full path
full_path = os.path.join(documents_path, filename)

# Save DataFrame
night_loc_df.to_csv(full_path, index=False)

print(f"Success! Saved to: {full_path}")

Success! Saved to: C:\Users\ajgraham/Documents\sexualassault_2018-2023.csv
